In [1]:
import numpy as np
import pandas as pd
import pickle

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_json('/Users/laurenfinkelstein/Metis/Projects/Project 3/credit_payment_default/results-20190430-210539.json')

# Data cleaning

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# is there only one entry per unique user id?
# answer = no duplicates

new = df.drop_duplicates()
new.count()
# df_filtered.groupby('id').count()
# df_filtered['id'].value_counts() > 1

In [3]:
# remove unecessary columns and reorder necessary columns

new_columns = ['default_payment_next_month', 'age', 'sex', 'marital_status', 'education_level',  
               'bill_amt_1', 'bill_amt_2', 'bill_amt_3', 'bill_amt_4', 'bill_amt_5', 'bill_amt_6', 
               'pay_0', 'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6', 
               'pay_amt_1', 'pay_amt_2', 'pay_amt_3', 'pay_amt_4', 'pay_amt_5', 'pay_amt_6', 
               'limit_balance']

model_data = df[new_columns]

In [ ]:
model_data.head()

In [ ]:
# figuring out what the different variables mean
# pay_0 etc. have additional labels other than labels described in GCP (e.g., 0, -1), but leave as is?

temp_columns = ['default_payment_next_month', 'age', 'sex', 'marital_status', 'education_level',  
               'bill_amt_1',
               'pay_0', 
               'pay_amt_1',
               'limit_balance']

df_temp = model_data[temp_columns]
df_temp.head()
# df_temp.pay_0.value_counts()

In [ ]:
# where pay_columns have a value < 0, change to 0
# ASSUMPTION: if pay columns have a negative value, customer paid on time

pay_columns = ['pay_0', 'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6']

for column in pay_columns:
    model_data.loc[model_data[column] < 0, column] = 0

In [ ]:
model_data.head()

In [ ]:
# sex: replace female=2 with female=0 to prepare for creating dummy variables
# or rename as 'male' and 'female'?

# model_data['sex'] = model_data['sex'].map({1: 'male', 2: 'female'})
# model_data.sex

model_data['sex2'] = np.where(model_data['sex'] > 1, 0, model_data['sex'])

# def gender_convert(num):
#     if num == 1:
#         return 'male'
#     if num == 2:
#         return 'female'
# model_data['sex'].map(gender_convert)

In [ ]:
model_data['sex2'].value_counts()

In [ ]:
model_data.columns

In [ ]:
model_data.drop(columns=['sex'], axis=1, inplace=True)
model_data.columns

In [ ]:
# replace marital_status number representations with categorical labels to prepare for dummy encoding

model_data['marital_status'] = model_data['marital_status'].map({1: 'married', 2: 'single', 3: 'others'})
model_data.marital_status

In [ ]:
# replace education level numerical label with string to prepare for dummy variables
# Education Level (1=graduate school, 2=university, 3=high school, 4=others, 5=unknown, 6=unknown)

model_data['education_level'] = model_data['education_level'].map({1: 'graduate_school', 2: 'university', 3: 'high school', 4: 'others', 5: 'unknown', 6: 'unknown'})
model_data.education_level

In [ ]:
with open ('cleaned_cc_default_data', 'wb') as f:
    pickle.dump(model_data, f)